In [2]:
import os
import csv
from pathlib import Path
from urllib.request import urlopen, urlretrieve
from urllib.parse import urlparse, urljoin

from bs4 import BeautifulSoup

In [12]:
def get_absolute_url(base_url: str, source: str) -> str:
    is_absolute = source.startswith(("http://", "https://"))
    has_external_domain = is_absolute and base_url not in source

    if has_external_domain:
        return None

    if is_absolute:
        url = source.replace("www.", "")
    elif source.startswith("www."):
        url = urljoin("http://", source.replace("www.", ""))
    else:
        url = urljoin(base_url, source)

    parsed = urlparse(url)
    return f"{parsed.scheme}://{parsed.netloc}{parsed.path}"


def get_download_path(base_url: str, absolute_url: str, download_dir: str):
    if not base_url.endswith("/"):
        base_url += "/"

    relative_path = absolute_url.replace(base_url, "")
    download_path = Path(download_dir) / relative_path

    if not os.path.exists(download_path.parent):
        os.makedirs(download_path.parent)

    return download_path

In [ ]:
download_dir = "downloaded"
base_url = "http://pythonscraping.com"
url = "http://www.pythonscraping.com"

html = urlopen(url)
bs = BeautifulSoup(html, "html.parser")
download_list = bs.find_all(src=True)

for download in download_list:
    if file_url := get_absolute_url(base_url, download["src"]):
        print(file_url)
        download_path = get_download_path(base_url, file_url, download_dir)
        urlretrieve(file_url, download_path)

In [3]:
with open("test.csv", "w+") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(("number", "number plus 2", "number times 2"))
    for i in range(10):
        writer.writerow((i, i + 2, i * 2))

In [10]:
import sqlite3

try:
    conn = sqlite3.connect("scraping.db")
    cursor = conn.cursor()

    with open("./create_table.sql", "r") as sqlFile:
        create_table_command = sqlFile.read()

    cursor.execute(create_table_command)
    conn.commit()
finally:
    cursor.close()
    conn.close()

In [ ]:
import smtplib
from email.mime.text import MIMEText

msg = MIMEText("The body of the email is here")

msg["Subject"] = "An Email Alert"
msg["From"] = "ryan@pythonscraping.com"
msg["To"] = "webmaster@pythonscraping.com"

s = smtplib.SMTP("localhost")
s.send_message(msg)
s.quit()

In [ ]:
import smtplib
from email.mime.text import MIMEText
from bs4 import BeautifulSoup
from urllib.request import urlopen
import time


def sendMail(subject, body):
    msg = MIMEText(body)
    msg["Subject"] = subject
    msg["From"] = "christmas_alerts@pythonscraping.com"
    msg["To"] = "ryan@pythonscraping.com"

    s = smtplib.SMTP("localhost")
    s.send_message(msg)
    s.quit()


bs = BeautifulSoup(urlopen("https://isitchristmas.com/"), "html.parser")
while bs.find("a", {"id": "answer"}).attrs["title"] == "NO":
    print("It is not Christmas yet.")
    time.sleep(3600)
    bs = BeautifulSoup(urlopen("https://isitchristmas.com/"), "html.parser")
sendMail(
    "It's Christmas!",
    "According to http://itischristmas.com, it is Christmas!",
)